In [1]:
import altair as alt
import polars as pl

from altair_utils import color_collision_ped_bike

In [5]:
df = pl.read_csv(r"Q:\CMP\LOS Monitoring 2024\safety-collisions.csv")

In [50]:
long_df = (
    df.unpivot(
        index=["year", "provisional"],
        on=["injury-pedestrian", "injury-bicycle", "fatal-pedestrian", "fatal-bicycle"],
        value_name="collisions",
    )
    .with_columns(
        pl.col("variable").str.split("-").list.to_struct(fields=["severity", "mode"]),
        date=pl.date(pl.col("year"), 1, 1),
        year=pl.col("year").cast(str),
    )
    .with_columns(
        year=pl.when(pl.col("provisional"))
        .then(pl.col("year") + "*")
        .otherwise(pl.col("year"))
    )
    .unnest("variable")
)

In [ ]:
injury_chart = (
    alt.Chart(
        long_df.filter((pl.col("year") > "2017") & (pl.col("severity") == "injury"))
    )
    .mark_bar()
    .encode(
        x="year:O",
        y=alt.Y("collisions", title="injury collisions"),
        color=alt.Color("mode:N").scale(
            domain=["pedestrian", "bicycle"], range=color_collision_ped_bike
        ),
        tooltip=["year", "provisional", "severity", "mode", "collisions"],
    )
    .interactive()
)
injury_chart.properties(width=250).save(
    "output/Links/safety-collisions-trend-injury.png"
)
injury_chart.properties(width="container").save(
    "output/Links/safety-collisions-trend-injury.html"
)
injury_chart.properties(width=250)

In [ ]:
fatal_chart = (
    alt.Chart(
        long_df.filter((pl.col("year") > "2017") & (pl.col("severity") == "fatal"))
    )
    .mark_bar()
    .encode(
        x="year:O",
        y=alt.Y("collisions", title="fatal collisions"),
        color=alt.Color("mode:N").scale(
            domain=["pedestrian", "bicycle"], range=color_collision_ped_bike
        ),
        tooltip=["year", "provisional", "severity", "mode", "collisions"],
    )
    .interactive()
)
fatal_chart.properties(width=250).save("output/Links/safety-collisions-trend-fatal.png")
fatal_chart.properties(width="container").save(
    "output/Links/safety-collisions-trend-fatal.html"
)
fatal_chart.properties(width=250)